In [3]:
import requests, re
import pandas as pd
import numpy as np

from nltk.stem import WordNetLemmatizer

In [26]:
data_w_freqs_path = '../data/pilot-classElicitation-free-4/class-elicitation-free-4-cleaned-data-w-sub2.csv'
data_w_freqs =  pd.read_csv(data_w_freqs_path)
# data_w_freqs

data_org = pd.read_csv('../data/pilot-classElicitation-free-4/class-elicitation-free-4-pilot-trials.csv')
data_org

data_path = '../data/pilot-classElicitation-free-4/class-elicitation-free-4-modal-responses.csv'
data = pd.read_csv(data_path) # plural NPs at cleanMode
data = data[['degree', 'superordinate', 'np_match', 'adj', 'cleanMode']]
# plural frequencies 
data_pl = pd.read_csv('../data/pilot-classElicitation-free-4/webgram-freqs-unique-nps.csv')
data_pl = data_pl.rename(columns={"NPs":"cleanMode"})

In [27]:
data

,degree,superordinate,np_match,adj,cleanMode
0,strength,people,adult,weak,adults
1,strength,people,adult,weak,adults
2,strength,people,adult,weak,adults
3,weight,people,adult,light,adults
4,weight,people,adult,light,adults
5,speed,animals,cheetah,quick,animals
6,speed,animals,cheetah,quick,animals
7,speed,animals,cheetah,quick,animals
8,speed,animals,cheetah,quick,animals
9,speed,animals,cheetah,quick,animals


In [41]:
# align original data with plural frequencies 
df_org_pl = data.join(data_pl.set_index('cleanMode'), on = 'cleanMode', how= 'inner', lsuffix = '_caller', rsuffix= '_new')
df_org_pl_super = data.join(data_pl.set_index('cleanMode'), on = 'superordinate', how= 'inner', lsuffix = '_caller', rsuffix= '_new')
df_org_pl_super = df_org_pl_super.rename(columns={'Frequencies': 'superFreq'})
df_pl = df_org_pl.join(df_org_pl_super.set_index('superordinate'), on = 'superordinate', how= 'inner', lsuffix = '_caller', rsuffix= '_new')
df_pl = df_pl.drop_duplicates()
df_pl = df_pl[['degree_caller', 'superordinate', 'cleanMode_caller', 'Frequencies', 'superFreq']]
df_pl = df_pl.drop_duplicates()
df_pl['fraction'] = df_pl.apply(lambda x: x['Frequencies']/x['superFreq'], axis = 1)
df_pl.sort_values('fraction', ascending = False)

,degree_caller,superordinate,cleanMode_caller,Frequencies,superFreq,fraction
118,darkness,times day,days,305158140,8984,33966.845503
299,darkness,times day,nights,21265731,8984,2367.067119
332,height,athletes,people,480323270,7268360,66.084133
339,speed,athletes,people,480323270,7268360,66.084133
35,price,alcoholic drinks,beer,20488618,377118,54.329462
126,temperature,seasons,days,305158140,12035578,25.354673
483,price,alcoholic drinks,wines,8940774,377118,23.708160
441,length_duration,readings,stories,115510308,8678675,13.309671
270,price,alcoholic drinks,liquor,4727755,377118,12.536540
472,darkness,times day,times of day,99907,8984,11.120548


In [7]:
# df= data_w_freqs.groupby(['degree', 'superordinate', 'NP'], as_index=False).drop_duplicates()
# ['subFreq'].sum()
# compute mean prior freq by scale-item pair
# df['mean_sub_freq'] = df.apply()
# df = df.to_frame()
np_org = data_org['np'].drop_duplicates()
df = data_w_freqs[['degree', 'superordinate', 'NP', 'subFreq', 'superFreq']].drop_duplicates()

df2 = df[['degree', 'superordinate', 'superFreq']].drop_duplicates()
df['fraction'] = df.apply(lambda x: x['subFreq']/x['superFreq'], axis = 1)
# df['fraction_super_mean'] = df.apply(lambda x: np.exp(0.2*np.log(x['subFreq']))/np.exp(0.2*np.log(x['superFreq'])), axis = 1)
# df['fraction_post'] = df.apply(lambda x: np.exp(0.2*np.log(x['subFreq']))/np.exp(0.2*np.log(x['superFreq'])), axis = 1)
df['frctMean'] = df['fraction']
df.n = df.groupby(['degree', 'superordinate'], as_index = False)['frctMean'].sum()
df.n2 = df.n.join(df2.set_index('superordinate'), on = 'superordinate', lsuffix='_caller', rsuffix='_other')
df.n2['prop_super'] = df.n2.apply(lambda x: x['frctMean']/3, axis = 1) # raw proportion means within item per scale for singular superordinates and subordinates, partly substituted 
df.n2.sort_values('prop_super', ascending = False)

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  from ipykernel import kernelapp as app


,degree_caller,superordinate,frctMean,degree_other,superFreq,prop_super
9,price,alcoholic drink,529.159064,price,158194,176.386355
10,price,statue,12.796950,price,4753368,4.265650
12,size,plant,3.085522,size,53249198,1.028507
19,weight,animal,2.175922,size,62579572,0.725307
19,weight,animal,2.175922,weight,62579572,0.725307
19,weight,animal,2.175922,loudness,62579572,0.725307
19,weight,animal,2.175922,speed,62579572,0.725307
19,weight,animal,2.175922,height,62579572,0.725307
7,loudness,animal,2.028679,size,62579572,0.676226
7,loudness,animal,2.028679,speed,62579572,0.676226


In [61]:

# read in unique responses
data_path = '../data/pilot-classElicitation-free-4/class-elicitation-free-4-modal-responses.csv'
data = pd.read_csv(data_path)

#manually include all the plural seed nouns, since they are in singular form in the data
pluralNouns = ["crows", "parrots", "seagulls", "carpets", "tiles", "woods", "chocolate", "jolly ranchers", "marshmallows", "birds", "giraffes", "monkeys",
"basketballers", "golfers", "jockeys", "bassetts", "chihuahuas", "dachshunds", "novels", "poems", "stories", "cats", "dogs", "donkeys", "auditoria", "classrooms", "study halls", "studyhalls",
"study-halls", "beers", "liquors", "wines", "bronze", "statues", "plastic", "platinum", "elephants", "mice", "bushes", "flowers", "trees", "cheetahs", "sloths", "joggers", "runners", "walkers",
"adults", "children", "teenagers", "hurricanes", "rains", "thunderstorms", "ice cream", "salads", "kids", "soups", "fall days", "summer days", "winter days", "fish", "babies", "caves", "caverns", "fox holes",
"downtown streets", "highways", "side roads", "times", "bronze statues", "plastic statues", "platinum statues",
              ]

# extract unique superordinates
supers = data['superordinate'].drop_duplicates().tolist()
# extract unique modal responses
modes = data['cleanMode'].drop_duplicates().tolist()
# all responses
# originalResps = data['cleanedPhrase'].apply(lambda x: x.replace("_", "")).drop_duplicates().tolist()
unique_nps_pl = list(set(supers + modes + pluralNouns))



98

In [53]:
lemmatizer = WordNetLemmatizer()

super_sg = data['superordinate'].drop_duplicates().apply(lambda word: lemmatizer.lemmatize(word)).tolist()
modes_sg = data['cleanMode'].drop_duplicates().apply(lambda word: lemmatizer.lemmatize(word)).tolist()

nouns_sg = pd.DataFrame(pluralNouns)[0].apply(lambda word: lemmatizer.lemmatize(word)).tolist()
# print(nouns_sg, super_sg, modes_sg)

unique_nps_sg = list(set(super_sg + modes_sg + nouns_sg))
unique_nps_sg

['seagull',
 'bronze',
 'cave',
 'mouse',
 'jockey',
 'season',
 'runner',
 'decoration',
 'kid',
 'elephant',
 'statue',
 'cavern',
 'salad',
 'plant',
 'times of day',
 'platinum',
 'chocolate',
 'athlete',
 'wood',
 'hurricane',
 'study halls',
 'day',
 'ice creams',
 'chihuahua',
 'flower',
 'downtown streets',
 'hole',
 'people',
 'cheetah',
 'fox holes',
 'child',
 'basketballers',
 'side roads',
 'studyhalls',
 'storm',
 'cat',
 'jogger',
 'sweet',
 'animal',
 'tile',
 'monkey',
 'alcoholic drinks',
 'fish',
 'food',
 'ice cream',
 'bassetts',
 'dachshund',
 'bird',
 'times day',
 'bottles of wine',
 'novel',
 'reading',
 'donkey',
 'night',
 'baby',
 'summer days',
 'story',
 'wine',
 'liquor',
 'rain',
 'crow',
 'bush',
 'beer',
 'fall days',
 'study-halls',
 'basketball players',
 'jolly ranchers',
 'plastic',
 'giraffe',
 'floor',
 'dog',
 'walker',
 'golfer',
 'road',
 'marshmallow',
 'sloth',
 'soup',
 'highway',
 'room',
 'teenager',
 'tree',
 'classroom',
 'auditorium',


In [ ]:
# try querying the original noun phrases 
np_original = data

In [ ]:

freqs_sg = []
words_sg = []
freqs_pl = []
words_pl = []
# construct the url
# for np in unique_nps_pl:
#     webgram_page = requests.get(base_url + "Web1T5_freq.perl?query=" + np + "&mode=Search&limit=50&threshold=100&optimize=on&wildcards=listed+normally&fixed=shown&.cgifields=optimize&.cgifields=debug")
#     word_soup = BeautifulSoup(webgram_page.text, 'html.parser')
#     #gives all the tables
#     tables = word_soup.find_all("table")
#     # get frequencies table (last one on the page)
#     freq_table = tables[len(tables)-1]
#     # get tags where the target information is
#     tds = freq_table.find_all("td")
#     text = []
#     for t in tds:
#         # get frequency and word
#         if t.text != "":
#             text.append(t.text)
#             print(text)
#     # the text list includes the frequency and the word
#     if len(text) == 0 :
#         continue
#     else:
#         freqs_pl.append(text[0])
#         words_pl.append(text[1])

# create DF of frequencies and words
# df_nps = pd.DataFrame(list(zip(words_pl, freqs_pl)), columns=['NPs', 'Frequencies'])
# df_nps.to_csv('../data/pilot-classElicitation-free-4/webgram-freqs-all-unique-nps.csv')

for np in unique_nps_sg:
    webgram_page = requests.get(base_url + "Web1T5_freq.perl?query=" + np + "&mode=Search&limit=50&threshold=100&optimize=on&wildcards=listed+normally&fixed=shown&.cgifields=optimize&.cgifields=debug")
    word_soup = BeautifulSoup(webgram_page.text, 'html.parser')
    #gives all the tables
    tables = word_soup.find_all("table")
    # get frequencies table (last one on the page)
    freq_table = tables[len(tables)-1]
    # get tags where the target information is
    tds = freq_table.find_all("td")
    text = []
    for t in tds:
        # get frequency and word
        if t.text != "":
            text.append(t.text)
            print(text)
    # the text list includes the frequency and the word
    if len(text) == 0 :
        continue
    else:
        freqs_sg.append(text[0])
        words_sg.append(text[1])

# create DF of frequencies and words
df_nps = pd.DataFrame(list(zip(words_sg, freqs_sg)), columns=[ 'NPs_SG', 'Frequencies_SG'])
df_nps.to_csv('../data/pilot-classElicitation-free-4/webgram-freqs-unique-nps-pl-sg.csv')
